In [15]:
import os
import csv
from collections import defaultdict

folder_path = '/Users/udz/Desktop/codons_analysis'
clusters_path = 'clusters_probabilities'
transcripts_pats = 'transcripts_probabilities'
merge_result_path = 'clusters_transcriptes_probabilities_cereus_mappings'
bacteria_name = 'cereus'

clusters_result_file_path = os.path.join(folder_path, clusters_path, bacteria_name + '_clusters_probabilities.csv')
transcripts_result_file_path = os.path.join(folder_path, transcripts_pats, bacteria_name + '.fna_probabilities.csv')

In [16]:
mappings = {
    'metUGG': {
        'CCC': 'CCA',
        'CCT': 'CCA',
        'CCG': 'CCA'
    },
    'metUAC': {
        'GTG': 'GTA',
        'GTT': 'GTA'
    },
    'ICG_ACG': {
        'CGC': 'CGT',
        'CGA': 'CGT',
        'CGG': 'CGT'
    }
}

In [17]:
clusters_data = defaultdict(lambda: defaultdict(int))

with open(clusters_result_file_path, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        clusters_data[row['codon1']][row['codon2']] = row['probability']

In [18]:
transcripts_data = defaultdict(lambda: defaultdict(int))

with open(transcripts_result_file_path, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        transcripts_data[row['codon1']][row['codon2']] = row['probability']

In [20]:
for file_suffix in mappings:
    merge_result_file_path = os.path.join(folder_path, merge_result_path, bacteria_name + '_' + file_suffix + '.csv')
    file_mapping = mappings[file_suffix]

    with open(merge_result_file_path, 'w') as f:
        w = csv.writer(f)
        w.writerow(['codon1', 'codon2', 'transcripts_probability', 'clusters_probability'])
        written_codons = set()
    
        for codon1 in transcripts_data:
            for codon2 in transcripts_data[codon1]:
                if file_mapping.get(codon1) and clusters_data[codon1]:
                    from_codon_1 = file_mapping.get(codon1)
                    w.writerow([codon1, codon2, transcripts_data[codon1][codon2], clusters_data[from_codon_1][codon2]])
                elif file_mapping.get(codon2) and clusters_data[codon2]:
                    from_codon_2 = file_mapping.get(codon2)
                    w.writerow([codon1, codon2, transcripts_data[codon1][codon2], clusters_data[codon1][from_codon_2]])
                else:
                    w.writerow([codon1, codon2, transcripts_data[codon1][codon2], clusters_data[codon1][codon2]])